This notebook will show how to compute the BGG cohomology of a module. 

First we import the necessary classes from the `bggcomplex` package. To do this we need to add the path to `bggcomplex` to the system path.

In [14]:
import sys
sys.path.append('../bggcomplex')

from bggcomplex import BGGComplex
from fast_module import FastLieAlgebraCompositeModule, FastModuleFactory, BGGCohomology

Next we create an instance of `BGGComplex`. The argument of this class is a string describing a root system. In this case we chose $\mathrm A_3$. For now the following root systems are small enough so that we can actually do computatons with them. This list may expand as the implementation is improved.
- $\mathrm A_n$ for $n\leq 4$
- $\mathrm B_n,\, \mathrm C_n,\,\mathrm D_n,\,$ for $n\leq 3$
- $\mathrm G_2$

In [15]:
BGG = BGGComplex('A3')

Next we instantiate a `FastModuleFactory`, which takes a Lie algebra $\mathfrak g$ as input. This should be the same Lie algebra as that of the BGG complex, and so we pass `BGG.LA`. 

In [16]:
factory = FastModuleFactory(BGG.LA)

Let's say we want to work with the module $\wedge^3 \mathfrak u$ with the coadjoint action.

We want to build this module out of smaller 'building blocks'. In this case the only 'building block' we need is the module $\mathfrak u\subset \mathfrak g$, which has an $\mathfrak n$-action by the coadjoint action. Therefore we call `factory.build_component('u','coad')`. We store the resulting module in a dictionary `component_dic` which will be later passed to `factory`.

In [17]:
component_dic = {'u':factory.build_component('u','coad')}

Next we instantiate a `FastLieAlgebraCompositeModule` with three arguments. The first is a `FastModuleFactory` instance, the second is the structure of the module in terms of some building blocks, and finally is a dictionary describing the building blocks. 

In this case the structure of $\wedge^3\mathfrak u$ is encoded as `[[("u",3,'wedge')]]`. The outer list is a list of direct sum components, the inner list a tensor components. The inner list is a list of 3-tuples. The first entry is a key of `component_dic` describing which building block to use, the second is a tensor power, and the third argument describes whether to use $\wedge^n$ (`'wedge'`) or $\mathrm{Sym}^n$ (`'sym'`).

For example $(\mathfrak u\otimes\mathfrak u)\oplus (\mathrm{Sym}^3\mathfrak u)$ would be encoded instead by `[[("u",1,'wedge'),("u",1,'wedge')],[('u',3,'sym')]] `. Note that $\wedge^1$ and $\mathrm{Sym}^1$ are equivalent.

In [18]:
components = [[("u",3,'wedge')]]

module = FastLieAlgebraCompositeModule(factory, components, component_dic)

Finally to compute the cohomology we create an instance of `BGGCohomology`, which takes as argument a `BGGComplex` and a `FastLieAlgebraCompositeModule`.

Then to compute the cohomology in degree $i$ we call `BGGCohomology.cohomology`. If we are using a jupyter notebook like now, we can instead call `BGGCohomology.cohomology_LaTeX`, which gives a bit more structured output. This function has as optional argument `complex_string`, which describes what will be shown between the brackets of $\mathrm H^i(\ldots)=$.

In [19]:
cohom = BGGCohomology(BGG, module)

cohom.cohomology_LaTeX(complex_string=r'\wedge^3\mathfrak u')

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The right hand side indicates which highest weight representations occur in the cohomology, and with which multiplicity. N.B. the powers $L(\dots)^n$ indicate direct sum power (not tensor powers).

In addition to $\mathfrak u\subset \mathfrak g$ there are a number of additional modules. We list all the options currently supported here:
- `'g'`, $\mathfrak g$, the entire Lie algebra.
- `'n'`, $\mathfrak n$, the subalgbera spanned by the $f_I$
- `'u'`, $\mathfrak u$, the subalgebra spanned by the $e_I$
- `'p'`, $\mathfrak p$, the parabolic subalgebra (need to specify a subset of simple roots)
- `'h'`, $\mathfrak h$, the torus spanned by the $h_i$
- `'b+'`, $\mathfrak b^+ = \mathfrak u\oplus \mathfrak h$, the positive Borel subalgebra
- `'b'`, $\mathfrak b = \mathfrak n\oplus \mathfrak h$, the (negative) Borel subalgebra

Most of them have a parabolic counterpart. To obtain these we supply an additional `subset` option, indicating the subset of the roots to remove. For example we can take `factory.build_component('u','coad',subset=[1,2])` which is spanned by the $e_I$ such that $I\nsubseteq \{\alpha_1,\alpha_2\}$.